In [4]:
import torch
import torch.nn as nn
from torch.optim import SGD 
import numpy as np

# Упражнение, для реализации "Ванильной" RNN
* Попробуем обучить сеть восстанавливать слово hello по первой букве. т.е. построим charecter-level модель

In [2]:
a = torch.ones((3,3))*3
b = torch.ones((3,3))*5

In [3]:
a @ b

tensor([[45., 45., 45.],
        [45., 45., 45.],
        [45., 45., 45.]])

In [4]:
a * b

tensor([[15., 15., 15.],
        [15., 15., 15.],
        [15., 15., 15.]])

In [5]:
#word = 'ololoasdasddqweqw123456789'
word = 'hello'

## Датасет. 
Позволяет:
* Закодировать символ при помощи one-hot
* Делать итератор по слову, которыей возвращает текущий символ и следующий как таргет

In [9]:
class WordDataSet:
    
    def __init__(self, word):
        self.chars2idx = {}
        self.indexs  = []
        for c in word: 
            if c not in self.chars2idx:
                self.chars2idx[c] = len(self.chars2idx)
                
            self.indexs.append(self.chars2idx[c])
            
        self.vec_size = len(self.chars2idx)
        self.seq_len  = len(word)
        
    def get_one_hot(self, idx):
        x = torch.zeros(self.vec_size)
        x[idx] = 1
        return x
    
    def __iter__(self):
        return zip(self.indexs[:-1], self.indexs[1:])
    
    def __len__(self):
        return self.seq_len
    
    def get_char_by_id(self, id):
        for c, i in self.chars2idx.items():
            if id == i: return c
        return None

## Реализация базовой RNN
<br/>
Скрытый элемент
$$ h_t= tanh⁡ (W_{ℎℎ} h_{t−1}+W_{xh} x_t) $$
Выход сети

$$ y_t = W_{hy} h_t $$

In [10]:
class VanillaRNN(nn.Module):
    
    def __init__(self, in_size=5, hidden_size=3, out_size=5):
        super(VanillaRNN, self).__init__()        
        self.x2hidden    = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden      = nn.Linear(in_features=hidden_size, out_features=hidden_size)
        self.activation  = nn.Tanh()
        self.outweight   = nn.Linear(in_features=hidden_size, out_features=out_size)
    
    def forward(self, x, prev_hidden):
        hidden = self.activation(self.x2hidden(x) + self.hidden(prev_hidden))
#         Версия без активации - может происходить gradient exploding
#         hidden = self.x2hidden(x) + self.hidden(prev_hidden)
        output = self.outweight(hidden)
        return output, hidden

## Инициализация переменных 

In [11]:
ds = WordDataSet(word=word)
rnn = VanillaRNN(in_size=ds.vec_size, hidden_size=3, out_size=ds.vec_size)
criterion = nn.CrossEntropyLoss()
e_cnt     = 100
optim     = SGD(rnn.parameters(), lr = 0.1, momentum=0.9)

# Обучение

In [12]:
CLIP_GRAD = True

for epoch in range(e_cnt):
    hh = torch.zeros(rnn.hidden.in_features)
    loss = 0
    optim.zero_grad()
    for sample, next_sample in ds:
        x = ds.get_one_hot(sample).unsqueeze(0)
        target =  torch.LongTensor([next_sample])

        y, hh = rnn(x, hh)
        
        loss += criterion(y, target)
     

    loss.backward()
    
    if epoch % 10 == 0:
        print (loss.data.item())
        if CLIP_GRAD: print("Clip gradient : ", torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=5))
    else: 
        if CLIP_GRAD: torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=1)
            
#     print("Params : ")
#     num_params = 0
#     for item in rnn.parameters():
#         num_params += 1
#         print(item.grad)
#     print("NumParams :", num_params)
#     print("Optimize")
    
    optim.step()

75.65586853027344
Clip gradient :  9.037430287291892
61.293922424316406
Clip gradient :  5.72250215665632
39.16627502441406
Clip gradient :  7.987201680228297
33.41114807128906
Clip gradient :  15.554054783908999
30.11669921875
Clip gradient :  30.98313973044023
23.993629455566406
Clip gradient :  8.121352114724939
22.80459976196289
Clip gradient :  3.601258526216851
20.942699432373047
Clip gradient :  20.9339665010292
20.418643951416016
Clip gradient :  9.798837868404721
25.220539093017578
Clip gradient :  20.939154339951006


# Тестирование

In [13]:
rnn.eval()
hh = torch.zeros(rnn.hidden.in_features)
id = 0
softmax  = nn.Softmax(dim=1)
predword = ds.get_char_by_id(id)
for c in enumerate(word[:-1]):
    x = ds.get_one_hot(id).unsqueeze(0)
    y, hh = rnn(x, hh)
    y = softmax(y)
    m, id = torch.max(y, 1)
    id = id.data[0]
    predword += ds.get_char_by_id(id)
print ('Prediction:\t' , predword)
print("Original:\t", word)
assert(predword == word)

Prediction:	 ol3asdasdasdasdasdasdasdas
Original:	 ololoasdasddqweqw123456789


AssertionError: 

# ДЗ
Реализовать LSTM и GRU модули, обучить их предсказывать тестовое слово
Сохранить ноутбук с предсказанием и пройденным assert и прислать на почту a.murashev@corp.mail.ru
c темой:


[МФТИ\_2019\_1] ДЗ №8 ФИО

In [14]:
#тестовое слово
word = 'ololoasdasddqweqw123456789'

## Реализовать LSTM

In [15]:
#Написать реализацию LSTM и обучить предсказывать слово

class LSTM(nn.Module):
    def __init__(self, in_size=5, hidden_size=4, out_size=5):
        
        super(LSTM, self).__init__()
        self.tg = nn.Tanh()
        self.sigm = nn.Sigmoid()
        
        self.hidden_c1 = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden_c2 = nn.Linear(in_features=hidden_size, out_features=hidden_size)
        
        self.hidden_i1 = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden_i2 = nn.Linear(in_features=hidden_size, out_features=hidden_size)
        
        self.hidden_f1 = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden_f2 = nn.Linear(in_features=hidden_size, out_features=hidden_size)
        
        self.hidden_o1 = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden_o2 = nn.Linear(in_features=hidden_size, out_features=hidden_size)
        
        self.out = nn.Linear(in_features=hidden_size, out_features=out_size)   
        
    def forward(self, x, prev_hidden, c_prev):
        
        c1 = self.tg(self.hidden_c1(x) + self.hidden_c2(prev_hidden))
        
        i_t = self.sigm(self.hidden_i1(x) + self.hidden_i2(prev_hidden))
        f_t = self.sigm(self.hidden_f1(x) + self.hidden_f2(prev_hidden))
        o_t = self.sigm(self.hidden_o1(x) + self.hidden_o2(prev_hidden))
        
        c_t = f_t * c_prev + i_t * c1
        
        h_t = o_t * self.tg(c_t)
        
        output = self.out(h_t)
        return output, h_t, c_t
        
        

In [16]:
ds = WordDataSet(word=word)
rnn = LSTM(in_size=ds.vec_size, hidden_size=9, out_size=ds.vec_size)
criterion = nn.CrossEntropyLoss()
e_cnt     = 100
optim     = SGD(rnn.parameters(), lr = 0.1, momentum=0.9)

In [17]:
CLIP_GRAD = True

for epoch in range(e_cnt):
    hh = torch.zeros(rnn.hidden_c2.in_features)
    cc = torch.zeros(rnn.hidden_c2.in_features)
    loss = 0
    optim.zero_grad()
    for sample, next_sample in ds:
        x = ds.get_one_hot(sample).unsqueeze(0)
        target =  torch.LongTensor([next_sample])

        y, hh, cc= rnn(x, hh, cc)
        
        loss += criterion(y, target)
     

    loss.backward()
    
    if epoch % 10 == 0:
        print (loss.data.item())
        if CLIP_GRAD: print("Clip gradient : ", torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=5))
    else: 
        if CLIP_GRAD: torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=1)
            
#     print("Params : ")
#     num_params = 0
#     for item in rnn.parameters():
#         num_params += 1
#         print(item.grad)
#     print("NumParams :", num_params)
#     print("Optimize")
    
    optim.step()

70.44019317626953
Clip gradient :  2.786715893973465
64.65707397460938
Clip gradient :  4.633944375709918
43.626731872558594
Clip gradient :  5.103064783137436
25.378124237060547
Clip gradient :  10.487058235377521
20.349328994750977
Clip gradient :  12.75321666721947
15.041677474975586
Clip gradient :  12.244897330710518
10.256861686706543
Clip gradient :  16.095181728100517
21.546659469604492
Clip gradient :  10.457370768837295
9.739273071289062
Clip gradient :  5.910449916242078
5.077642440795898
Clip gradient :  4.509234551109054


In [18]:
rnn.eval()
hh = torch.zeros(rnn.hidden_c2.in_features)
cc = torch.zeros(rnn.hidden_c2.in_features)
id = 0
softmax  = nn.Softmax(dim=1)
predword = ds.get_char_by_id(id)
for c in enumerate(word[:-1]):
    x = ds.get_one_hot(id).unsqueeze(0)
    y, hh, cc= rnn(x, hh, cc)
    y = softmax(y)
    m, id = torch.max(y, 1)
    id = id.data[0]
    predword += ds.get_char_by_id(id)
print ('Prediction:\t' , predword)
print("Original:\t", word)
assert(predword == word)

Prediction:	 ololoasdasddqweqw123456789
Original:	 ololoasdasddqweqw123456789


## Реализовать GRU

In [19]:
#Написать реализацию GRU и обучить предсказывать слово

class GRU(nn.Module):
    def __init__(self, in_size=5, hidden_size=3, out_size=5):
        super(GRU, self).__init__()
        #реализуем
        self.tg = nn.Tanh()
        
        self.sigm = nn.Sigmoid()
        
        self.hidden_u1 = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden_u2 = nn.Linear(in_features=hidden_size, out_features=hidden_size)
        
        self.hidden_r1 = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden_r2 = nn.Linear(in_features=hidden_size, out_features=hidden_size)
        
        self.hidden_h1 = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden_h2 = nn.Linear(in_features=hidden_size, out_features=hidden_size)  
        
        self.out = nn.Linear(in_features=hidden_size, out_features=out_size) 
        
    def forward(self, x, prev_hidden):
        
        u_t = self.sigm(self.hidden_u1(x) + self.hidden_u2(prev_hidden))
        r_t = self.sigm(self.hidden_r1(x) + self.hidden_r2(prev_hidden))
        h_1 = self.tg(self.hidden_h1(x) + self.hidden_h2(r_t * prev_hidden))
        h_t = ((1 - u_t) * h_1) + (u_t * prev_hidden)
        output = self.out(h_t)
        return output, h_t

In [20]:
ds = WordDataSet(word=word)
rnn = GRU(in_size=ds.vec_size, hidden_size=8, out_size=ds.vec_size)
criterion = nn.CrossEntropyLoss()
e_cnt     = 100
optim     = SGD(rnn.parameters(), lr = 0.1, momentum=0.9)

In [21]:
CLIP_GRAD = True

for epoch in range(e_cnt):
    hh = torch.zeros(rnn.hidden_u2.in_features)
    loss = 0
    optim.zero_grad()
    for sample, next_sample in ds:
        x = ds.get_one_hot(sample).unsqueeze(0)
        target =  torch.LongTensor([next_sample])

        y, hh = rnn(x, hh)
        
        loss += criterion(y, target)
     

    loss.backward()
    
    if epoch % 10 == 0:
        print (loss.data.item())
        if CLIP_GRAD: print("Clip gradient : ", torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=5))
    else: 
        if CLIP_GRAD: torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=1)
            
#     print("Params : ")
#     num_params = 0
#     for item in rnn.parameters():
#         num_params += 1
#         print(item.grad)
#     print("NumParams :", num_params)
#     print("Optimize")
    
    optim.step()

72.17572021484375
Clip gradient :  5.611296591065968
60.54824447631836
Clip gradient :  5.1479021241908764
31.222505569458008
Clip gradient :  7.360420938949959
11.752055168151855
Clip gradient :  4.728708588445481
4.589115619659424
Clip gradient :  1.3891959886566019
1.5938000679016113
Clip gradient :  0.9175795743113586
0.23430633544921875
Clip gradient :  0.26942385093208143
0.08988666534423828
Clip gradient :  0.08379358722633924
0.05649566650390625
Clip gradient :  0.04637134830203998
0.04168510437011719
Clip gradient :  0.03004707088338469


In [22]:
### rnn.eval()
hh = torch.zeros(rnn.hidden_u2.in_features)
id =0
softmax  = nn.Softmax(dim=1)
predword = ds.get_char_by_id(id)
for c in enumerate(word[:-1]):
    x = ds.get_one_hot(id).unsqueeze(0)
    y, hh = rnn(x, hh)
    y = softmax(y)
    m, id = torch.max(y, 1)
    id = id.data[0]
    predword += ds.get_char_by_id(id)
print ('Prediction:\t' , predword)
print("Original:\t", word)
assert(predword == word)

Prediction:	 ololoasdasddqweqw123456789
Original:	 ololoasdasddqweqw123456789
